# Contexto

## Objetivo

Este projeto foi desenvolvido como parte do desafio do **IADT — Fase 1**. O objetivo é desenvolver modelos preditivos para estimar os **custos médicos individuais** cobrados por seguros de saúde, com base em atributos como:

- Idade
- Índice de massa corporal (IMC)
- Número de filhos
- Tabagismo
- Sexo
- Região geográfica

Trata-se de um problema de **regressão supervisionada**, com variável alvo contínua (custos médicos), abordado com modelos lineares e baseados em árvores de decisão.


## Fonte de Dados

O conjunto de dados foi obtido na plataforma **Kaggle**, disponível no repositório ["Healthcare Insurance"](https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance).


## Modelos Utilizados

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regressor
- Random Forest Regressor
- Gradient Boosting Regressor
- Ordinary Least Squares (OLS)


# Setup

## Instalação de Dependências

In [ ]:
# Instalação das bibliotecas necessárias para o projeto
! pip install --quiet pandas numpy matplotlib seaborn scikit-learn statsmodels kagglehub joblib

## Importação de Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Modelagem
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Pré-processamento
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# Validação e avaliação
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

# Estatística
import statsmodels.api as sm

# KaggleHub (carregamento de dataset)
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Salvamento e carregamento de modelos treinados
import joblib

In [ ]:
# Carrega o dataset usando kagglehub

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "willianoliveiragibin/healthcare-insurance",
    "insurance.csv",
)

assert not df.empty, "Dataset não foi carregado corretamente"

# Exploração de Dados

Nesta etapa, buscamos compreender a estrutura e características principais do conjunto de dados. A análise inclui:

- Dimensões do dataset
- Tipos de variáveis
- Valores ausentes
- Estatísticas descritivas
- Visualizações iniciais para facilitar a interpretação inicial


In [ ]:
# Retorna a dimensionalidade do DataFrame.
df.shape

💡 O dataset possui 1338 linhas (registros) e 7 colunas (variáveis).


In [ ]:
# Exibe informações gerais do dataset: tipos de dados, contagem de valores não nulos e uso de memória.
df.info()

💡 O dataset não possui valores nulos, o que indica que ele já foi previamente limpo.

As variáveis categóricas são:

- `sex`
- `smoker`
- `region`

As variáveis numéricas são:

- `age`
- `bmi`
- `children`
- `charges`


In [ ]:
# Visualiza as 5 primeiras linhas do dataset para inspecionar amostras reais dos dados.
df.head()

In [ ]:
# Gera estatísticas descritivas para variáveis numéricas do dataset.
df.describe()

💡 A análise descritiva revela informações importantes sobre as variáveis numéricas.

Essas métricas permitem avaliar a simetria, a presença de outliers e a dispersão das variáveis — informações úteis para decisões de pré-processamento e modelagem.

| **Métrica**   | **Descrição**                                                  | **Exemplo**                                                                            |
| ------------- | -------------------------------------------------------------- | -------------------------------------------------------------------------------------- |
| `count`       | Número de valores não nulos na coluna.                         | Todas as colunas têm **1338 entradas**, indicando que o dataset está completo.         |
| `mean`        | Média aritmética dos valores.                                  | A média de IMC (`bmi`) é aproximadamente **30**, indicando uma tendência ao sobrepeso. |
| `std`         | Desvio padrão — indica o grau de dispersão em relação à média. | O desvio padrão de `charges` é elevado, mostrando grande variação nos custos médicos.  |
| `min` / `max` | Menor e maior valor observado na variável.                     | A idade (`age`) varia de **18 a 64 anos**, cobrindo uma população adulta.              |
| `25%`         | Primeiro quartil — 25% dos dados estão abaixo desse valor.     | 25% dos pacientes têm menos de **27 anos** (`age`), indicando uma população jovem.     |
| `50%`         | Mediana — ponto central da distribuição.                       | A mediana de `charges` é cerca de **9.400**, menor que a média, sugerindo assimetria.  |
| `75%`         | Terceiro quartil — 75% dos dados estão abaixo desse valor.     | 75% têm `charges` abaixo de **17.000**, indicando que os valores mais altos são raros. |


In [ ]:
# Exibe estatísticas descritivas para variáveis categóricas.
df.describe(include=['object'])

💡 As estatísticas descritivas das variáveis categóricas revelam que:

- As colunas `sex` e `smoker` são binárias, com apenas duas categorias distintas.
- A coluna `region` possui 4 categorias, o que a caracteriza como categórica nominal.
- O campo `top` indica a categoria mais comum em cada variável, e `freq` mostra sua frequência — útil para entender o balanceamento das classes.


### Análises gráficas das variáveis

In [ ]:
# Cria gráficos de barras para visualizar a frequência das categorias em 'sex', 'smoker' e 'region'.

fig, axs = plt.subplots(1, 3, figsize=(15, 4))

sns.countplot(data=df, x="sex", ax=axs[0])
axs[0].set_title("Distribuição de sex")

sns.countplot(data=df, x="smoker", ax=axs[1])
axs[1].set_title("Distribuição de smoker")

sns.countplot(data=df, x="region", ax=axs[2])
axs[2].set_title("Distribuição de region")

plt.tight_layout()
plt.show()

💡 A distribuição das variáveis categóricas mostra que:

- **`sex`**: A variável está aproximadamente balanceada entre masculino e feminino, o que evita viés de gênero no treinamento do modelo.
- **`smoker`**: A maior parte dos indivíduos são não-fumantes, indicando desequilíbrio de classes.
- **`region`**: As quatro regiões (`northeast`, `southeast`, `southwest`, `northwest`) têm distribuições similares, o que é positivo para generalização do modelo.

Essas análises ajudam a antecipar a necessidade de tratamento para desbalanceamentos (como em `smoker`) e a decidir o tipo de codificação adequada.


In [ ]:
# Gera histogramas para visualizar a distribuição das variáveis numéricas: age, bmi, children e charges.

def plot_histogram(df, column, ax=None, kde=True):
    sns.histplot(df[column], kde=kde, ax=ax)
    ax.set_title(f"Distribuição de {column}")

cols = ["age", "bmi", "children", "charges"]
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
for col, ax in zip(cols, axs.ravel()):
    kde_flag = False if col == "children" else True
    plot_histogram(df, col, ax, kde=kde_flag)
plt.tight_layout()
plt.show()

💡 A análise da distribuição das variáveis numéricas mostra que:

- **`age`**: A distribuição é ligeiramente assimétrica, o que pode influenciar o modelo a focar em custos de faixas etárias mais jovens.
- **`bmi`**: A distribuição é quase normal, mas com uma cauda à direita, indicando alguns casos de obesidade.
- **`children`**: A maioria dos indivíduos tem 0 ou poucos filhos, com poucos casos com 4 ou mais filhos.
- **`charges`**: A distribuição é muito assimétrica e possui uma cauda longa, indicando a presença de outliers — alguns poucos pacientes têm custos médicos extremamente altos.


In [ ]:
# Gera boxplots para inspecionar a distribuição e detectar outliers nas variáveis numéricas.

def plot_boxplot(df, column, ax=None):
    sns.boxplot(x=df[column], ax=ax)
    ax.set_title(f"Boxplot de {column}")

fig, axs = plt.subplots(2, 2, figsize=(15, 8))
for col, ax in zip(cols, axs.ravel()):
    plot_boxplot(df, col, ax)
plt.tight_layout()
plt.show()


💡 Os boxplots das variáveis numéricas revelam:

- **`age`**: Distribuição relativamente simétrica, sem outliers aparentes. A mediana está próxima ao centro da faixa etária.
- **`bmi`**: Leve assimetria à direita e presença de alguns outliers com IMC elevado, o que pode indicar indivíduos obesos.
- **`children`**: Como variável discreta, o boxplot mostra saltos em valores fixos. Não há outliers, mas a distribuição é claramente enviesada para baixo.
- **`charges`**: Forte assimetria à direita com diversos outliers, o que confirma a cauda longa observada no histograma. Esses valores extremos podem afetar o desempenho de modelos mais sensíveis à variância.


### Análises gráficas das correlações

In [ ]:
# Cria gráficos de dispersão e stripplots para examinar relações com 'charges'
fig, axs = plt.subplots(3, 2, figsize=(15, 12))

# Dispersão: idade vs. custo
sns.scatterplot(data=df, x="age", y="charges", ax=axs[0, 0])
axs[0, 0].set_title("charges vs. age")

# Dispersão: IMC vs. custo
sns.scatterplot(data=df, x="bmi", y="charges", ax=axs[0, 1])
axs[0, 1].set_title("charges vs. bmi")

# Dispersão: número de filhos vs. custo
sns.scatterplot(data=df, x="children", y="charges", ax=axs[1, 0])
axs[1, 0].set_title("charges vs. children")

# Stripplot: fumante vs. custo
sns.stripplot(data=df, x="smoker", y="charges", ax=axs[1, 1], jitter=True, alpha=0.5)
axs[1, 1].set_title("charges vs. smoker")

# Stripplot: sexo vs. custo
sns.stripplot(data=df, x="sex", y="charges", ax=axs[2, 0], jitter=True, alpha=0.5)
axs[2, 0].set_title("charges vs. sex")

# Stripplot: região vs. custo
sns.stripplot(data=df, x="region", y="charges", ax=axs[2, 1], jitter=True, alpha=0.5)
axs[2, 1].set_title("charges vs. region")

plt.tight_layout()
plt.show()


💡 Os gráficos de dispersão e stripplots mostram padrões importantes entre variáveis explicativas e os custos médicos (`charges`):

- `age`: tendência de aumento nos custos com a idade, reforçando sua relevância como variável contínua.
- `bmi`: disperso, mas com acúmulo de custos elevados para valores acima de 30 (obesidade).
- `children`: pouca relação visual direta.
- `smoker`: impacto evidente — fumantes concentram-se em uma faixa muito mais alta de `charges`.
- `sex`: pouca relação visual direta.
- `region`: variações discretas nos custos médios entre regiões.

Essas visualizações reforçam a necessidade de considerar **efeitos não lineares e interações entre variáveis**, especialmente ao escolher modelos como árvores de decisão e ensemble methods.


In [ ]:
# Gera gráficos de barras para o custo médico médio por grupo de variáveis

df_viz = df.copy()
df_viz["age_group"] = pd.cut(df_viz["age"], bins=range(15, 70, 5))
df_viz["bmi_group"] = pd.cut(df_viz["bmi"], bins=range(15, 55, 5))

# Define colunas categóricas e rótulos para exibição
group_configs = {
    "age_group": ("Faixa etária", "Custo médio por faixa etária"),
    "bmi_group": ("Faixa de IMC", "Custo médio por faixa de IMC"),
    "children": ("Número de filhos", "Custo médio por número de filhos"),
    "smoker": ("Fumante", "Custo médio por tabagismo"),
    "sex": ("Sexo", "Custo médio por sexo"),
    "region": ("Região", "Custo médio por região")
}

# Define número de subplots dinamicamente com base na quantidade de variáveis
fig, axs = plt.subplots(3, 2, figsize=(15, 12))

for ax, (col, (xlabel, title)) in zip(axs.ravel(), group_configs.items()):
    grouped = df_viz.groupby(col)["charges"].mean().reset_index()
    sns.barplot(data=grouped, x=col, y="charges", ax=ax)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Custo médio (USD)")

plt.tight_layout()
plt.show()


💡 A análise dos custos médicos médios por grupos revela padrões claros e úteis para interpretação:

- `age`: Há um aumento progressivo no custo médio conforme a faixa etária avança, refletindo o aumento de risco com o envelhecimento.
- `bmi`: A média de `charges` tende a subir a partir de IMCs acima de 30, indicando o impacto da obesidade.
- `children`: Embora o padrão não seja tão forte, há uma tendência leve de aumento com mais filhos.
- `smoker`: Diferença gritante — fumantes têm, em média, custos médicos muito superiores aos não fumantes.
- `sex`: Pequena diferença entre os grupos, mas consistente: indivíduos do sexo masculino tendem a apresentar médias ligeiramente superiores.
- `region`: As diferenças entre regiões são sutis, mas notáveis em alguns agrupamentos — o que pode indicar influências geográficas nos custos.

Essa visualização por média facilita a comunicação dos efeitos médios das variáveis de forma mais acessível.

# Pré-processamento de Dados

Nesta etapa, realizaremos transformações para preparar os dados para os modelos de regressão.


In [ ]:
# Fazemos uma cópia do DataFrame original para aplicar codificações sem alterar os dados brutos
df_encoded = df.copy()

In [ ]:
# Codifica as variáveis binárias 'sex' e 'smoker' manualmente.
df_encoded["sex"] = df_encoded["sex"].map({"female": 0, "male": 1})
df_encoded["smoker"] = df_encoded["smoker"].map({"no": 0, "yes": 1})

In [ ]:
# Aplica codificação one-hot na variável 'region', criando colunas binárias para representar cada região.
df_encoded = pd.get_dummies(df_encoded, columns=["region"], drop_first=False)

In [ ]:
# Converte colunas booleanas para inteiros (0 ou 1), garantindo compatibilidade com o modelos do statsmodels, como o OLS.
df_encoded = df_encoded.astype({col: int for col in df_encoded.select_dtypes('bool').columns})

In [ ]:
df_encoded.head()

In [ ]:
# Gera um mapa de calor com a correlação entre variáveis numéricas para identificar relações lineares com o custo médico.
plt.figure(figsize=(10, 6))
sns.heatmap(df_encoded.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Mapa de Correlação")
plt.show()

💡 A matriz de correlação revela:

- `smoker` tem a maior correlação com  (0.79), indicando forte impacto nos custos médicos — possivelmente devido a maior risco associado ao tabagismo.
- `age` possui correlação moderada com  (0.30), o que faz sentido, já que idosos tendem a ter mais despesas médicas.
- `bmi` também mostra uma correlação positiva (0.20), o que também faz sentido, especialmente considerando casos de obesidade.
- `sex`, `children` e as variáveis de `region` têm correlações fracas (próximas de 0), sugerindo pouca influência direta em `charges`.

Essas informações ajudam a entender quais variáveis são mais relevantes para o modelo e quais podem ter influência limitada.

Mesmo assim, manter as regiões codificadas pode ajudar modelos mais complexos a capturar interações não-lineares.


In [ ]:
# Cria duas versões do dataset: uma para modelos lineares e outra para modelos baseados em árvore.
df_tree = df_encoded.copy()
df_linear = df_encoded.drop(columns=["region_southwest"])

ℹ️ Ao aplicar one-hot encoding na variável categórica `region`, são criadas múltiplas colunas binária.

No entanto, manter todas essas colunas pode causar **multicolinearidade** em modelos lineares — uma condição em que variáveis altamente correlacionadas afetam negativamente os coeficientes da regressão.

Para evitar isso, removemos uma das colunas (no caso, `region_southwest`).

In [ ]:
# Normaliza variáveis numéricas no dataset linear
scaler = StandardScaler()
numeric_features = ["age", "bmi", "children"]
df_linear[numeric_features] = scaler.fit_transform(df_linear[numeric_features])

ℹ️ Modelos como **Regressão Linear**, **Ridge** e **Lasso** são sensíveis à escala das variáveis.

Isso significa que atributos com valores maiores podem influenciar desproporcionalmente os coeficientes.

Para evitar esse problema, utilizamos o **StandardScaler**, que transforma as variáveis numéricas (`age`, `bmi`, `children`) para uma distribuição com média 0 e desvio padrão 1.  

Essa padronização garante que todos os atributos tenham a mesma importância inicial no processo de ajuste dos coeficientes.


In [ ]:
df_linear.head()

# Modelagem

Nesta etapa, definimos um dicionário chamado `models` contendo os algoritmos de regressão que serão avaliados. Cada entrada inclui:

- O modelo propriamente instanciado com seus parâmetros padrão (exceto o `random_state`, para garantir reprodutibilidade);
- O conjunto de dados correspondente, preparado de forma adequada:
  - Modelos lineares (`Linear Regression`, `Ridge`, `Lasso`) utilizam o `df_linear`, que passou por normalização e ajuste de multicolinearidade;
  - Modelos baseados em árvore (`Decision Tree`, `Random Forest`, `Gradient Boosting`) utilizam o `df_tree`, sem necessidade de normalização.

Essa separação permite comparar diferentes algoritmos de forma justa, utilizando os dados no formato mais apropriado para seu funcionamento.

In [ ]:
models = {
    "Linear Regression": {
        "model": LinearRegression(),
        "data": df_linear
    },
    "Ridge": {
        "model": Ridge(),
        "data": df_linear
    },
    "Lasso": {
        "model": Lasso(),
        "data": df_linear
    },
    "Decision Tree": {
        "model": DecisionTreeRegressor(random_state=42),
        "data": df_tree
    },
    "Random Forest": {
        "model": RandomForestRegressor(random_state=42, n_jobs=-1),
        "data": df_tree
    },
    "Gradient Boosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "data": df_tree
    },
}

# Treinamento e Avaliação do Modelo

Nesta etapa, treinamos os modelos para prever os custos médicos com base nos dados de treino.
Depois, avaliamos seu desempenho em dados de teste — que os modelos ainda não conheciam.

Utilizamos métricas como **R²**, **MSE** e **RMSE** para medir a qualidade das previsões.

## Avaliação com Holdout (Treino/Teste)

Nesta etapa, usamos a estratégia **holdout** — uma divisão simples dos dados em treino e teste.

É uma abordagem eficiente para avaliação inicial, mas pode gerar resultados enviesados dependendo da divisão feita.

In [ ]:
results_holdout = {} # Armazena as métricas por modelo
y_preds_holdout = {} # Armazena os valores previstos por modelo

for model_name, config in models.items():
    model = config["model"]
    df_base = config["data"]

    # Divide os dados em X (features) e y (target)
    X = df_base.drop("charges", axis=1)
    y = df_base["charges"]

    # Divide em conjunto de treino e teste (80/20), com random_state para reprodutibilidade
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Treina o modelo
    model.fit(X_train, y_train)

    # Gera predições para o conjunto de teste
    y_pred = model.predict(X_test)

    # Calcula métricas de avaliação
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calcula RMSE manualmente (compatível com versões mais antigas do sklearn)

    # Salva as métricas no dicionário de resultados
    results_holdout[model_name] = {
        "R²": r2,
        "RMSE": rmse,
    }

    # Armazena predições e valores reais para análises gráficas posteriores
    y_preds_holdout[model_name] = y_pred

df_results_holdout = (pd.DataFrame(results_holdout).T.sort_values(by="R²", ascending=False))
display(df_results_holdout)

| **Métrica** | **Nome**                      | **Descrição**                                                   | **Interpretação**                                 |
| ----------- | ----------------------------- | --------------------------------------------------------------- | ------------------------------------------------- |
| `R²`        | Coeficiente de Determinação   | Mede o quanto da variação em `charges` é explicada pelo modelo. | Varia de 0 a 1. Quanto maior, melhor.             |
| `MSE`       | Erro Quadrático Médio         | Média dos erros elevados ao quadrado.                           | Penaliza fortemente grandes erros.                |
| `RMSE`      | Raiz do Erro Quadrático Médio | Raiz quadrada do MSE.                                           | Tem a mesma unidade da variável alvo (`charges`). |

In [ ]:
# Gera gráfico de dispersão: valores reais vs. previstos
def plot_prediction_vs_actual(model_name, ax, y_true, y_pred):
    ax.scatter(y_true, y_pred, alpha=0.5)
    
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label="Referência (y = x)")
    
    ax.set_xlabel("Valores Reais")
    ax.set_ylabel("Valores Previstos")
    ax.set_title(f"Real vs. Previsão ({model_name})")
    ax.grid(True)
    ax.legend()

fig, axs = plt.subplots(1, 1, figsize=(15, 4))

plot_prediction_vs_actual("Linear Regression", axs, y_test, y_preds_holdout["Linear Regression"])

plt.tight_layout()
plt.show()

## Avaliação com Validação Cruzada (K-Fold)

Nesta etapa, avaliamos os modelos utilizando validação cruzada do tipo **K-Fold**, que divide os dados em 5 partes e roda múltiplas rodadas de treino e teste.

Essa abordagem gera métricas mais estáveis e menos dependentes de uma única divisão dos dados.

In [ ]:
from sklearn.model_selection import cross_validate

# Define o esquema de validação cruzada K-Fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Dicionário para armazenar os resultados
results_table_kfold = {}

# Loop por todos os modelos
for model_name, config in models.items():
    model = config["model"]
    df_base = config["data"]

    X = df_base.drop("charges", axis=1)
    y = df_base["charges"]

    # Realiza validação cruzada com múltiplas métricas
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=kfold,
        scoring={"r2": "r2", "mse": "neg_mean_squared_error"},
        return_train_score=False
    )

    # Calcula as métricas agregadas
    r2_scores = cv_results["test_r2"]
    mse_scores = -cv_results["test_mse"]
    rmse_scores = np.sqrt(mse_scores)

    # Armazena as métricas no dicionário
    results_table_kfold[model_name] = {
        "Mean R²": np.mean(r2_scores),
        "R² Std": np.std(r2_scores),
        "Mean MSE": np.mean(mse_scores),
        "MSE Std": np.std(mse_scores),
        "Mean RMSE": np.mean(rmse_scores),
        "RMSE Std": np.std(rmse_scores),
    }

# Converte os resultados em DataFrame ordenado por desempenho
df_results_kfold = (
    pd.DataFrame(results_table_kfold)
    .T
    .sort_values(by="Mean R²", ascending=False)
    .reset_index()
)
df_results_kfold.rename(columns={"index": "Model"}, inplace=True)
display(df_results_kfold)


## Ajuste de Hiperparâmetros com GridSearchCV

Nesta etapa, buscamos os melhores hiperparâmetros para cada modelo utilizando validação cruzada (K-Fold) e a métrica R² como critério de avaliação.

Isso nos permite melhorar a performance dos modelos além do ajuste padrão, testando diferentes combinações de parâmetros.

In [ ]:
# Dicionário com modelos e respectivos grids de hiperparâmetros
model_configs_grid = {
    "Linear Regression": {
        "estimator": models["Linear Regression"]["model"],
        "param_grid": {}
    },
    "Ridge": {
        "estimator": models["Ridge"]["model"],
        "param_grid": {
            "alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000]
        }
    },
    "Lasso": {
        "estimator": models["Lasso"]["model"],
        "param_grid": {
            "alpha": [0.001, 0.01, 0.1, 1, 10, 100]
        }
    },
    "Decision Tree": {
        "estimator": models["Decision Tree"]["model"],
        "param_grid": {
            "max_depth": [3, 5, 10],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 3]
        }
    },
    "Random Forest": {
        "estimator": models["Random Forest"]["model"],
        "param_grid": {
            "n_estimators": [100, 200],
            "max_depth": [5, 10, None],
            "min_samples_leaf": [1, 3],
            "max_features": ["sqrt", "log2"]
        }
    },
    "Gradient Boosting": {
        "estimator": models["Gradient Boosting"]["model"],
        "param_grid": {
            "n_estimators": [100, 200],
            "learning_rate": [0.03, 0.05, 0.1],
            "max_depth": [2, 3, 4],
            "min_samples_leaf": [1, 3],
            "subsample": [0.8, 1.0]
        }
    }
}

# Avaliação dos modelos com GridSearchCV + validação cruzada
results_grid = []

for model_name, config in model_configs_grid.items():
    print(f"Ajustando hiperparâmetros: {model_name}")

    # Seleciona o DataFrame correto com base no modelo
    df_base = models[model_name]["data"]
    X = df_base.drop("charges", axis=1)
    y = df_base["charges"]

    # Executa a busca em grade com validação cruzada
    grid_search = GridSearchCV(
        estimator=config["estimator"],
        param_grid=config["param_grid"],
        scoring={"r2": "r2", "mse": "neg_mean_squared_error"},
        refit="r2",
        cv=kfold,
        n_jobs=-1,
        return_train_score=True
    )

    grid_search.fit(X, y)

    # Armazena os resultados principais
    results_grid.append({
        "Model": model_name,
        "Best R² (CV)": grid_search.best_score_,
        "Best Parameters": grid_search.best_params_,
        "Best Estimator": grid_search.best_estimator_
    })

# Cria o DataFrame ordenado por desempenho
df_results_grid = (
    pd.DataFrame(results_grid)
    .sort_values(by="Best R² (CV)", ascending=False)
    .reset_index(drop=True)
)
display(df_results_grid.drop(columns=["Best Estimator"]))


# Validação Estatística

Nesta seção, realizamos uma análise estatística clássica do modelo de regressão linear utilizando a biblioteca `statsmodels`.

Diferente das abordagens anteriores, que avaliaram o desempenho preditivo dos modelos com métricas como R² e RMSE, aqui o objetivo é entender:
- Quais variáveis explicativas possuem efeito estatisticamente significativo sobre os custos médicos (`charges`);
- A magnitude e direção desses efeitos (coeficientes);
- A presença de multicolinearidade entre variáveis preditoras;
- Se os pressupostos da regressão linear (normalidade dos resíduos, homocedasticidade, etc.) são atendidos.

In [ ]:
# Adiciona constante para o modelo OLS (intercepto)
X_train_sm = sm.add_constant(X_train, has_constant='add')
X_test_sm = sm.add_constant(X_test, has_constant='add')

# Converte y para float64 (recomendado para o OLS)
y_train_sm = y_train.astype(float)
y_test_sm = y_test.astype(float)

# Treina modelo OLS com statsmodels
ols_model = sm.OLS(y_train_sm, X_train_sm).fit()

# Exibe um resumo estatístico completo do model OLS (coeficientes, p-values, R² ajustado, etc.)
ols_model.summary()

💡 Interpretação dos resultados do modelo OLS:

* **R² = 0.742**
  * Isso significa que o modelo consegue **explicar 74,2% da variação nos custos médicos** com base nas variáveis de entrada. Ou seja, boa parte dos fatores que fazem o custo variar estão sendo capturados pelas variáveis do modelo.

* **R² ajustado = 0.740**
  * É uma versão do R² que penaliza o uso excessivo de variáveis. Isso evita que o modelo pareça melhor só porque adicionamos mais colunas (mesmo que irrelevantes).
  * O fato de estar muito próximo do R² comum indica que as variáveis adicionadas realmente ajudam a explicar o custo — não estão apenas “inflando” a performance do modelo artificialmente.

* **F-statistic = 380.9 (p < 0.001)**
  * Esse teste verifica se o modelo, como um todo, é melhor do que simplesmente chutar a média para todos os casos.
  * O valor de **p < 0.001** indica que a chance de o modelo ser ruim por acaso é **menor que 0.1%** — o que é considerado altamente significativo em estatística.
  * Portanto, temos evidência forte de que o conjunto de variáveis realmente ajuda a prever os custos.


| Variável          | Coef.   | P-valor | Significativo? | Interpretação                                                           |
| ----------------- | ------- | ------- | -------------- | ----------------------------------------------------------------------- |
| **age**           | +256.98 | <0.001  | ✅              | Cada ano a mais aumenta o custo médio em \~R\$257.                     |
| **sex**           | -18.59  | 0.961   | ❌              | Sem efeito significativo entre homens/mulheres.                        |
| **bmi**           | +337.09 | <0.001  | ✅              | Cada ponto extra no IMC aumenta os custos em \~R\$337.                 |
| **children**      | +425.28 | 0.006   | ✅              | Cada filho a mais aumenta o custo médio em \~R\$425.                   |
| **smoker**        | +23.657 | <0.001  | ✅              | Fumantes têm em média R\$23.657 a mais em custos.                      |
| region\_northeast | +657.86 | 0.223   | ❌              | Não tem impacto significativo.                                         |
| region\_northwest | +287.19 | 0.598   | ❌              | Não tem impacto significativo.                                         |
| region\_southwest | -151.94 | 0.775   | ❌              | Não tem impacto significativo.                                         |


<!-- ### 🧪 **Resumo das variáveis significativas**

* ✅ **`smoker`** é de longe a variável com **maior impacto e significância**.
* ✅ **`age`**, **`bmi`** e **`children`** também são estatisticamente significativas.
* ❌ **`sex`** e **`region_*`** **não têm efeito significativo** — ou seja, não explicam variações relevantes nos custos médicos nesse modelo.

---

### ⚠️ Considerações finais

* A estrutura do modelo é robusta, com boa explicação da variabilidade de `charges`.
* **Não há multicolinearidade perfeita**, dado que o modelo foi ajustado com One-Hot Encoding (removendo uma dummy).
* A ausência de significância de algumas variáveis pode indicar que elas:

  * Não afetam diretamente os custos médicos
  * Têm efeito indireto já absorvido por outras variáveis (como `smoker` ou `bmi`)
* Você poderia optar por **remover variáveis com p > 0.05** para simplificar o modelo sem perda de desempenho.

---

Se quiser, posso gerar um gráfico com os coeficientes significativos ou auxiliar na simplificação do modelo OLS. Deseja isso? -->


# Conclusão

Este notebook teve como objetivo prever os custos médicos individuais (`charges`) com base em dados demográficos e comportamentais simulados, como idade, IMC, número de filhos, tabagismo e região. E encontramos os seguintes insights:

- A variável **`smoker`** apresentou o maior impacto nos custos, com efeito estimado entre R$ 22.735 e R$ 24.566.
- Os modelos lineares explicaram cerca de **74% da variação** em `charges`, enquanto o **Random Forest** atingiu **86%**, com menor erro médio (RMSE).
- Variáveis como **`sex`** e **`region`** não apresentaram efeito significativo isoladamente.
- A análise estatística com OLS forneceu maior interpretabilidade, enquanto o Random Forest teve melhor desempenho preditivo.
- O modelo Random Forest foi o que melhor capturou os padrões dos dados, sendo a escolha mais adequada para aplicações com foco em precisão.

# Extra

## Pipeline Final

Após a validação cruzada e a busca em grade por hiperparâmetros ideais, identificamos que o modelo com melhor desempenho foi o **Gradient Boosting Regressor**.

Para consolidar esse modelo em um fluxo reprodutível e escalável, construímos uma pipeline completa.
Esta pipeline realiza o pré-processamento dos dados e aplica o modelo treinado de forma integrada.

- **Codificação ordinal** para variáveis binárias (`sex`, `smoker`);
- **One-hot encoding** para a variável categórica `region` (com exclusão da primeira categoria para evitar multicolinearidade);
- **Passagem direta** das variáveis numéricas (`age`, `bmi`, `children`);
- **Treinamento e predição** com o `GradientBoostingRegressor`, configurado com os melhores hiperparâmetros obtidos via GridSearchCV.

Essa abordagem permite encapsular todo o processo de preparação e modelagem em um único objeto, facilitando tanto a manutenção quanto a aplicação do modelo em novos dados.

In [ ]:
# Definição das colunas
numeric_features = ['age', 'bmi', 'children']
binary_features = ['sex', 'smoker']
categorical_features = ['region']

# Pré-processador
preprocessor = ColumnTransformer(transformers=[
    ('encode_binary', OrdinalEncoder(dtype=int), binary_features),
    ('encode_categorical', OneHotEncoder(drop='first'), categorical_features),
    ('pass_numeric', 'passthrough', numeric_features)
])

# Pipeline final com o melhor modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        min_samples_leaf=1,
        subsample=1.0,
        random_state=42
    ))
])

# Divisão dos dados
X = df.drop(columns=['charges'])
y = df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)


In [ ]:
# Salva o pipeline treinado em um arquivo usando joblib
joblib.dump(pipeline, 'gradient_boosting_pipeline.pkl')

In [ ]:
# Carrega e usa o pipeline treinado do arquivo
pipeline_loaded = joblib.load('gradient_boosting_pipeline.pkl')

y_pred = pipeline_loaded.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'R²: {r2:.4f}')
print(f'RMSE: {rmse:.2f}')